# Importing the packages and data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import scipy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from scipy.special import gamma, digamma, polygamma
import math

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import random

import scipy.optimize
from scipy.optimize import Bounds

In [3]:
import sys
sys.path.append("../smote_cd/")
from smote_cd.dataset_generation import softmax

In [4]:
import dirichlet_regression

In [222]:
import dirichlet_regression_old

In [5]:
from scipy.optimize import fmin, newton, minimize

In [208]:
def create_features_matrices(n_samples,n_features,choice_W='X_dependent',threshold_neighbors=0.3,nneighbors=5):
    X = np.random.uniform(size=(n_samples,n_features))
    X = (X-X.min())/(X.max()-X.min())
    X = np.array([np.concatenate(([1],x)) for x in X])
    Z = np.random.uniform(size=(n_samples,n_features))
    
    if choice_W == 'random':
        neighbors = NearestNeighbors(n_neighbors=nneighbors).fit(np.arange(n_samples).reshape(-1,1))
        W = neighbors.kneighbors_graph(np.arange(n_samples).reshape(-1,1)).toarray()
    else:
        distance_matrix = scipy.spatial.distance_matrix(X,X)
        W = np.zeros(np.shape(distance_matrix))
        W[distance_matrix < threshold_neighbors] = 1
    # replace the 1 on the diagonal by 0
    np.fill_diagonal(W,0)
    # scaling the matrix, so that the sum of each row is 1
    W = W/W.sum(axis=1)[:,None]
    return(X,Z,W)

In [292]:
n_features = 2
n_classes = 3
n = 200

np.random.seed(21)

beta = np.array([[0.  , -1.6 , 1.],
                 [0.  , 1.8, -1.4],
                 [0.  , 1.4, -1.1 ]])

gamma_var = np.round(np.random.normal(size=(n_features)),2)
rho=0.1

beta0 = np.zeros((n_features+1,n_classes-1))
gamma0 = [0.5,0.5]
rho0 = [0.]

In [293]:
params0_ns = np.concatenate([beta0.flatten(),gamma0]) 
params0_s = np.concatenate([beta0.flatten(),gamma0,rho0]) 

In [216]:
X,Z,W = create_features_matrices(n,n_features,choice_W='random')

In [268]:
M = np.identity(n) - rho*W
phi = np.exp(np.matmul(Z,gamma_var))
alpha = mu*phi[:,None]
Y = np.array([np.random.dirichlet(alpha_i) for alpha_i in alpha])

In [286]:
%%time
solution_no_spatial = minimize(dirichlet_regression.objective_func_loglik_no_spatial, params0_ns, args=(X, Y, Z))

Wall time: 32.9 ms


In [287]:
print(solution_no_spatial.message)
beta_sol_ns = np.zeros((n_features+1, n_classes))
beta_sol_ns[:,1:] = solution_no_spatial.x[:(n_features+1)*(n_classes-1)].reshape((n_features+1, n_classes-1))
gamma_sol_ns = solution_no_spatial.x[(n_features+1)*(n_classes-1):]

Optimization terminated successfully.


In [289]:
print(beta)
print(beta_sol_ns)

[[ 0.  -1.6  1. ]
 [ 0.   1.8 -1.4]
 [ 0.   1.4 -1.1]]
[[ 0.         -1.72359183  0.98397595]
 [ 0.          1.56487304 -1.46687742]
 [ 0.          1.62626268 -1.31521836]]


In [290]:
print(gamma_var)
print(gamma_sol_ns)

[-0.05 -0.11]
[ 0.08009107 -0.26238328]


In [298]:
%%time
solution_spatial = minimize(dirichlet_regression.objective_func_loglik_spatial, params0_s, args=(X, Y, W, Z))

Wall time: 266 ms


In [299]:
print(solution_spatial.message)
beta_sol_s = np.zeros((n_features+1, n_classes))
beta_sol_s[:,1:] = solution_spatial.x[:(n_features+1)*(n_classes-1)].reshape((n_features+1, n_classes-1))
gamma_sol_s = solution_spatial.x[(n_features+1)*(n_classes-1):-1]
rho_sol_s = solution_spatial.x[-1]

Optimization terminated successfully.


In [300]:
print(beta)
print(beta_sol_s)

[[ 0.  -1.6  1. ]
 [ 0.   1.8 -1.4]
 [ 0.   1.4 -1.1]]
[[ 0.         -1.71252533  1.02280318]
 [ 0.          1.56127227 -1.46999705]
 [ 0.          1.62037141 -1.33603333]]


In [301]:
print(gamma_var)
print(gamma_sol_s)

[-0.05 -0.11]
[ 0.08895287 -0.27187074]


In [302]:
print(rho)
print(rho_sol_s)

0.1
0.06042223140470986


# Test speed with Maupiti data

In [316]:
X_maupiti = np.array(pd.read_csv('Data Dirichlet/X_maupiti_for_softmax.csv', header=None))
Y_maupiti = np.array(pd.read_csv('Data Dirichlet/Y_maupiti_for_softmax.csv',header=None))

In [318]:
n_features = 5
n_classes = 15

In [312]:
n_maupiti = len(X_maupiti)

In [314]:
train_ind, test_ind = train_test_split(np.arange(n_maupiti),test_size=0.25)

In [327]:
X_maupiti_1 = np.ones((n_maupiti,n_features+1))
X_maupiti_1[:,1:] = X_maupiti

In [328]:
X_train, X_test = X_maupiti_1[train_ind], X_maupiti_1[test_ind]
Y_train, Y_test = Y_maupiti[train_ind], Y_maupiti[test_ind]
#Z_train, Z_test = Z_maupiti[train_ind], Z_maupiti[test_ind]
#W_train, W_test = W_maupiti[train_ind][:,train_ind], W_maupiti[test_ind][:,test_ind]

In [338]:
beta_maupiti = np.zeros((1+n_features)*(n_classes-1))

rho_maupiti = [0.1]

params0_maupiti = np.concatenate([beta_maupiti.flatten(),rho_maupiti])
params0_maupiti_ns = beta_maupiti.flatten()

In [339]:
phi = np.ones(len(X_train))

In [340]:
%%time
solution_no_spatial = minimize(dirichlet_regression.objective_func_loglik_no_spatial, params0_maupiti_ns, 
                               args=(X_train, Y_train, X_train, phi))

Wall time: 2.8 s


In [341]:
solution_no_spatial.message

'Optimization terminated successfully.'